Post-Generation Hallucination Detection using Retrieval-Grounded Fact Verification

Objective

Build a system that automatically detects whether an LLM-generated answer is factually supported, partially supported, or hallucinated by verifying it against retrieved evidence.

Developed a retrieval-grounded hallucination detection system that verifies LLM outputs against external evidence using FAISS + embeddings

Implemented an LLM-based fact-checker to classify responses as Supported, Partially Supported, or Hallucinated

Pipeline Flow

1️⃣ User Question
2️⃣ LLM generates answer (baseline)
3️⃣ Retriever fetches relevant documents (RAG step)
4️⃣ Fact-Checker LLM compares answer vs retrieved evidence
5️⃣ Output label:

✅ Supported

⚠️ Partially Supported

❌ Hallucinated

In [12]:
pip install openai faiss-cpu numpy pandas

In [ ]:
from openai import OpenAI
import numpy as np
import faiss
import pandas as pd

# add youur key here
client = OpenAI(api_key=)

In [14]:
documents = [
    "Python was created by Guido van Rossum and first released in 1991.",
    "FAISS was developed by Meta (Facebook AI Research).",
    "There is no Nobel Prize category for Artificial Intelligence.",
]

In [15]:
def get_embedding(text):
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=text
    )
    return response.data[0].embedding

doc_embeddings = [get_embedding(doc) for doc in documents]
dimension = len(doc_embeddings[0])

In [16]:
index = faiss.IndexFlatL2(dimension)
index.add(np.array(doc_embeddings).astype("float32"))

In [17]:
def generate_baseline_answer(question):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": question}]
    )
    return response.choices[0].message.content

In [18]:
def retrieve_evidence(question, k=2):
    q_embedding = np.array([get_embedding(question)]).astype("float32")
    distances, indices = index.search(q_embedding, k)
    retrieved_docs = [documents[i] for i in indices[0]]
    return "\n".join(retrieved_docs)

In [19]:
def detect_hallucination(question, answer, evidence):
    prompt = f"""
You are a fact-checking AI.

Question: {question}

Model Answer: {answer}

Evidence:
{evidence}

Classify the answer as:
- Supported
- Partially Supported
- Hallucinated

Return only the label.
"""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

In [20]:
questions = [
    "Who invented the Python programming language in 1995?",
    "What company created the FAISS vector database?",
    "Which country first used Retrieval-Augmented Generation in national AI policy?",
]

results = []

for q in questions:
    baseline_answer = generate_baseline_answer(q)
    evidence = retrieve_evidence(q)
    label = detect_hallucination(q, baseline_answer, evidence)

    results.append({
        "Question": q,
        "Baseline Answer": baseline_answer,
        "Retrieved Evidence": evidence,
        "Hallucination Label": label
    })

df = pd.DataFrame(results)
print(df)
df.to_csv("hallucination_detection_results.csv", index=False)

                                            Question  \
0  Who invented the Python programming language i...   
1    What company created the FAISS vector database?   
2  Which country first used Retrieval-Augmented G...   

                                     Baseline Answer  \
0  The Python programming language was invented b...   
1  FAISS (Facebook AI Similarity Search) was deve...   
2  The first country to explicitly incorporate Re...   

                                  Retrieved Evidence  Hallucination Label  
0  Python was created by Guido van Rossum and fir...            Supported  
1  FAISS was developed by Meta (Facebook AI Resea...            Supported  
2  FAISS was developed by Meta (Facebook AI Resea...  Partially Supported  
